In [8]:
# Necessary imports
from langchain_community.tools import DuckDuckGoSearchRun, StructuredTool
from pydantic import BaseModel, Field
from langchain_classic.agents import create_react_agent, AgentExecutor
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_classic import hub
from dotenv import load_dotenv
import requests

# Load api key
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [2]:
# LLM config
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

# Molde config
model = ChatHuggingFace(llm=llm)

In [22]:
# Tools
search_tool = DuckDuckGoSearchRun()

# Input arguments schema
class WeatherTool(BaseModel):
    city: str = Field(required=True, description="Name of the city from where we fetch weather conditions.")

# Weather tool
def get_weather_conditions(city: str) -> str:
    """This is an tool which fetches the current weather conditions of the given city name."""    
    url = f"https://api.weatherapi.com/v1/current.json?key=626c2425693948fea60110858260601&q={city}"
    response = requests.get(url=url)

    return response.json()

# Tool creation
weather_tool = StructuredTool.from_function(
    func=get_weather_conditions,
    args_schema=WeatherTool
)

/var/folders/2n/vnp09xnn0s35m3xr_5c7_b300000gn/T/ipykernel_68811/3140403441.py:6: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  city: str = Field(required=True, description="Name of the city from where we fetch weather conditions.")


In [4]:
# Pull ReAct prompt from langchain
prompt = hub.pull("hwchase17/react")

In [28]:
# Agent for thinking & planning
agent = create_react_agent(
    llm=model,
    tools=[search_tool, weather_tool],
    prompt=prompt
)

# Agent executor for performing actual tasks
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, weather_tool],
    verbose=True
)

In [29]:
agent_response = agent_executor.invoke({"input": "What is the capital of Uttar Pradesh and its weather conditions."})
print(agent_response["output"])



> Entering new AgentExecutor chain...
Thought: I need to find the capital of Uttar Pradesh and then its weather conditions. 
Action: get_weather_conditions
Action Input: Lucknow{'location': {'name': 'Lucknow', 'region': 'Uttar Pradesh', 'country': 'India', 'lat': 26.85, 'lon': 80.917, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1767698825, 'localtime': '2026-01-06 16:57'}, 'current': {'last_updated_epoch': 1767698100, 'last_updated': '2026-01-06 16:45', 'temp_c': 14.0, 'temp_f': 57.2, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 6.3, 'wind_kph': 10.1, 'wind_degree': 299, 'wind_dir': 'WNW', 'pressure_mb': 1020.0, 'pressure_in': 30.12, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 94, 'cloud': 0, 'feelslike_c': 13.4, 'feelslike_f': 56.1, 'windchill_c': 20.0, 'windchill_f': 68.1, 'heatindex_c': 20.0, 'heatindex_f': 68.1, 'dewpoint_c': 4.4, 'dewpoint_f': 39.8, 'vis_km': 1.5, 'vis_miles': 0.0, 'uv': 0.3, 